In [ ]:
-------------------------------------------------PIPLINE3/SHEMAS LINKING + BART-------------------------------------------------

In [19]:
import re
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple, Optional

import spacy                  # pour NLP
from rapidfuzz import fuzz    # pour fuzzy matching (plus moderne que fuzzywuzzy)
import networkx as nx         # pour le graphe question + schéma


****************************I/PARTIE NLP****************************

In [1]:
import json
import re
import sqlite3
from pprint import pprint


In [5]:
BASE_DIR = "datafinal"

# === Charger train_spider.json ===
with open(f"{BASE_DIR}/train_spider.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)


In [7]:
# === Charger tables.json ===
with open(f"{BASE_DIR}/tables.json", "r", encoding="utf-8") as f:
    tables = json.load(f)

print("Nombre de schémas de DB :", len(tables))
print("Premier schéma :")
pprint(tables[0])


Nombre de schémas de DB : 166
Premier schéma :
{'column_names': [[-1, '*'],
                  [0, 'perpetrator id'],
                  [0, 'people id'],
                  [0, 'date'],
                  [0, 'year'],
                  [0, 'location'],
                  [0, 'country'],
                  [0, 'killed'],
                  [0, 'injured'],
                  [1, 'people id'],
                  [1, 'name'],
                  [1, 'height'],
                  [1, 'weight'],
                  [1, 'home town']],
 'column_names_original': [[-1, '*'],
                           [0, 'Perpetrator_ID'],
                           [0, 'People_ID'],
                           [0, 'Date'],
                           [0, 'Year'],
                           [0, 'Location'],
                           [0, 'Country'],
                           [0, 'Killed'],
                           [0, 'Injured'],
                           [1, 'People_ID'],
                           [1, 'Name'],
         

### Étape 1 – Récupérer le schéma d’une DB à partir de db_id

On va créer une fonction utilitaire pour trouver le bon schéma dans tables.json.

In [11]:
def get_db_schema(db_id, tables_json):
    """
    Retourne le schéma correspondant à un db_id donné,
    ou None si non trouvé.
    """
    for db in tables_json:
        if db["db_id"] == db_id:
            return db
    return None


Testons sur le premier exemple de train_spider :

In [14]:
example = train_data[0]
db_id = example["db_id"]
schema = get_db_schema(db_id, tables)

print("db_id de l'exemple :", db_id)
print("\nSchéma associé :")
pprint(schema)


db_id de l'exemple : department_management

Schéma associé :
{'column_names': [[-1, '*'],
                  [0, 'department id'],
                  [0, 'name'],
                  [0, 'creation'],
                  [0, 'ranking'],
                  [0, 'budget in billions'],
                  [0, 'num employees'],
                  [1, 'head id'],
                  [1, 'name'],
                  [1, 'born state'],
                  [1, 'age'],
                  [2, 'department id'],
                  [2, 'head id'],
                  [2, 'temporary acting']],
 'column_names_original': [[-1, '*'],
                           [0, 'Department_ID'],
                           [0, 'Name'],
                           [0, 'Creation'],
                           [0, 'Ranking'],
                           [0, 'Budget_in_Billions'],
                           [0, 'Num_Employees'],
                           [1, 'head_ID'],
                           [1, 'name'],
                           [1, 'bor

### Étape 2 – Afficher joli le schéma (pour bien le comprendre)

In [17]:
def pretty_print_schema(schema):
    """
    Affiche les tables et colonnes d'un schéma Spider
    de manière lisible.
    """
    table_names = schema["table_names"]
    col_names = schema["column_names"]  # liste [ (table_idx, col_name), ... ]

    print(f"📌 DB : {schema['db_id']}\n")

    print("Tables :")
    for i, t in enumerate(table_names):
        print(f"  [{i}] {t}")

    print("\nColonnes :")
    for (table_idx, col_name) in col_names:
        if table_idx == -1:  # l'entrée spéciale '*'
            continue
        table_name = table_names[table_idx]
        print(f"  - {table_name}.{col_name}")

# Test sur le schéma de l'exemple 0
pretty_print_schema(schema)


📌 DB : department_management

Tables :
  [0] department
  [1] head
  [2] management

Colonnes :
  - department.department id
  - department.name
  - department.creation
  - department.ranking
  - department.budget in billions
  - department.num employees
  - head.head id
  - head.name
  - head.born state
  - head.age
  - management.department id
  - management.head id
  - management.temporary acting


### Étape 3 – Préprocessing de la question

In [21]:
# Charger le modèle spaCy (anglais ici car Spider est en anglais)
# ⚠️ À faire UNE SEULE FOIS dans ton script
nlp = spacy.load("en_core_web_sm")

In [23]:
@dataclass
class TokenInfo:
    """
    Représente un token de la question avec plusieurs infos utiles
    pour le schema linking.
    """
    text: str        # forme originale
    lemma: str       # lemme (forme de base)
    pos: str         # catégorie grammaticale (NOUN, VERB, NUM, etc.)
    dep: str         # relation de dépendance
    head: str        # tête de dépendance (texte du token racine)
    idx: int         # position du token dans la phrase

In [25]:
@dataclass
class QuestionPreproc:
    """
    Structure de données contenant toutes les infos
    prétraitées pour une question.
    """
    raw: str                     # question originale
    normalized: str              # question normalisée (minuscule, etc.)
    tokens: List[TokenInfo]      # liste des tokens annotés
    values: List[str]            # valeurs candidates (nombres et strings)
    doc: Any                     # objet spaCy Doc (si besoin plus tard)

In [35]:
def normalize_question(text: str) -> str:
    """
    Normalise la question :
    - minuscule
    - nettoyage des espaces
    - suppression de toute la ponctuation
    """
    # minuscule
    text = text.lower()

    # supprimer toute la ponctuation
    # conserve lettres, chiffres et espaces
    text = re.sub(r"[^a-z0-9\s]", "", text)

    # réduire les espaces multiples
    text = re.sub(r"\s+", " ", text)

    return text.strip()


In [37]:
def extract_values_from_doc(doc: spacy.tokens.Doc) -> List[str]:
    """
    Extrait les valeurs candidates :
    - nombres (POS = NUM)
    - strings entre guillemets (ex : "texas")
    """
    values = []

    # 1) Nombres explicites dans le texte
    for token in doc:
        if token.pos_ == "NUM":
            values.append(token.text)

    # 2) Strings entre guillemets (simples ou doubles)
    text = doc.text
    string_values = re.findall(r'"([^"]+)"', text) + re.findall(r"'([^']+)'", text)
    values.extend(string_values)

    # enlever doublons en gardant l'ordre
    seen = set()
    unique_values = []
    for v in values:
        if v not in seen:
            seen.add(v)
            unique_values.append(v)

    return unique_values


In [39]:
def preprocess_question(question: str, nlp_model) -> QuestionPreproc:
    """
    Pipeline de préprocessing complet pour UNE question :
    - normalisation
    - analyse spaCy
    - extraction tokens + lemmes + POS + dépendances
    - extraction des valeurs candidates
    """
    raw = question
    normalized = normalize_question(raw)

    # Passage dans spaCy pour avoir POS, lemmes, dépendances, etc.
    doc = nlp_model(normalized)

    tokens_info: List[TokenInfo] = []
    for i, tok in enumerate(doc):
        tokens_info.append(
            TokenInfo(
                text=tok.text,
                lemma=tok.lemma_,
                pos=tok.pos_,
                dep=tok.dep_,
                head=tok.head.text,
                idx=i
            )
        )

    values = extract_values_from_doc(doc)

    return QuestionPreproc(
        raw=raw,
        normalized=normalized,
        tokens=tokens_info,
        values=values,
        doc=doc
    )


In [41]:
# ============================================================
# Exemple d'utilisation de l'étape 3 sur le premier exemple
# ============================================================
example = train_data[0]
question = example["question"]
print("Question originale :", question)

q_preproc = preprocess_question(question, nlp)

print("\nQuestion normalisée :", q_preproc.normalized)
print("Tokens :")
for t in q_preproc.tokens:
    print(f"  idx={t.idx:2d} text={t.text:10s} lemma={t.lemma:10s} pos={t.pos:5s} dep={t.dep:10s} head={t.head}")

print("\nValeurs candidates :", q_preproc.values)

Question originale : How many heads of the departments are older than 56 ?

Question normalisée : how many heads of the departments are older than 56
Tokens :
  idx= 0 text=how        lemma=how        pos=SCONJ dep=advmod     head=many
  idx= 1 text=many       lemma=many       pos=ADJ   dep=amod       head=heads
  idx= 2 text=heads      lemma=head       pos=NOUN  dep=nsubj      head=are
  idx= 3 text=of         lemma=of         pos=ADP   dep=prep       head=heads
  idx= 4 text=the        lemma=the        pos=DET   dep=det        head=departments
  idx= 5 text=departments lemma=department pos=NOUN  dep=pobj       head=of
  idx= 6 text=are        lemma=be         pos=AUX   dep=ROOT       head=are
  idx= 7 text=older      lemma=old        pos=ADJ   dep=acomp      head=are
  idx= 8 text=than       lemma=than       pos=ADP   dep=prep       head=older
  idx= 9 text=56         lemma=56         pos=NUM   dep=pobj       head=than

Valeurs candidates : ['56']


SpaCy attribue à chaque token un type grammatical comme :

NOUN → nom

VERB → verbe

ADJ → adjectif

ADV → adverbe

NUM → nombre

DET → déterminant ("the", "a")

ADP → adposition (prépositions comme "of", "than")

AUX → auxiliaire ("are", "is")

SCONJ → conjonction (souvent pour mots interrogatifs comme "how")

👉 POS sert à identifier la fonction grammaticale basique d'un mot.

DEP (Dependency Relation)

➡️ C’est la relation syntaxique entre ce token et son mot parent (HEAD).
Elle indique comment le mot s’intègre dans la phrase, par exemple :

nsubj → nom sujet (ex. "heads" est sujet de "are")

pobj → objet d’une préposition (ex. "departments" est objet de "of")

advmod → modificateur adverbial (ex. “how” modifie “many”)

amod → adjectif modifiant un nom (ex. “many” modifie “heads”)

prep → préposition ("of", "than")

acomp → adjectival complement (ex. “older” complète “are”)

ROOT → racine de la phrase (ex. “are”)

det → déterminant (ex. “the” → “departments”)

👉 DEP sert à comprendre la structure grammaticale complète de la phrase.

Dans l’étape 3, nous avons d’abord normalisé la question (minuscule, suppression de la ponctuation, nettoyage) pour obtenir un texte propre et cohérent. Ensuite, nous avons appliqué une analyse linguistique complète avec spaCy (lemmes, catégories grammaticales, dépendances et valeurs numériques) afin de préparer toutes les informations nécessaires pour le schema linking des étapes suivantes.

### Étape 4 – Baseline de Schema Linking (matching lexical)

1. Dataclasses pour représenter le schéma normalisé

In [43]:
from dataclasses import dataclass
from typing import List, Dict, Any


@dataclass
class ColumnInfo:
    """
    Représente une colonne dans une base Spider après normalisation.
    - table_idx : index de la table dans table_names
    - table_name : nom normalisé de la table (minuscule)
    - column_name : nom normalisé de la colonne (minuscule)
    - full_name : concaténation 'table.colonne'
    """
    table_idx: int
    table_name: str
    column_name: str
    full_name: str


@dataclass
class NormalizedSchema:
    """
    Schéma d'une base de données Spider après normalisation,
    prêt pour le schema linking.
    - db_id : identifiant de la base
    - table_names : liste des noms de tables normalisés (minuscule)
    - original_table_names : noms de tables originaux (pour info)
    - columns : liste d'objets ColumnInfo
    """
    db_id: str
    table_names: List[str]
    original_table_names: List[str]
    columns: List[ColumnInfo]


2. Fonction de normalisation du schéma

À partir du schéma brut de tables.json, on fabrique un NormalizedSchema.

In [45]:
def normalize_schema(schema: Dict[str, Any]) -> NormalizedSchema:
    """
    À partir d'un schéma Spider brut (une entrée de tables.json),
    crée une version normalisée pour le schema linking :
      - noms de tables en minuscule
      - noms de colonnes en minuscule
      - construction des noms complets "table.colonne"
    """

    db_id = schema["db_id"]

    # Noms originaux des tables (tels que dans tables.json)
    table_names_original = schema["table_names"]  # ex: ["department", "head", "management"]

    # Version normalisée : tout en minuscule
    table_names_norm = [t.lower() for t in table_names_original]

    columns_info: List[ColumnInfo] = []

    # schema["column_names"] = liste de [table_idx, col_name]
    for (table_idx, col_name) in schema["column_names"]:
        # table_idx = -1 correspond à la pseudo-colonne "*" globale -> on l'ignore
        if table_idx == -1:
            continue

        # nom de la table normalisée
        tname = table_names_norm[table_idx]

        # nom de la colonne normalisé
        col_norm = col_name.lower()   # ex: "budget in billions"

        # nom complet "table.colonne"
        full_name = f"{tname}.{col_norm}"

        columns_info.append(
            ColumnInfo(
                table_idx=table_idx,
                table_name=tname,
                column_name=col_norm,
                full_name=full_name
            )
        )

    return NormalizedSchema(
        db_id=db_id,
        table_names=table_names_norm,
        original_table_names=table_names_original,
        columns=columns_info
    )


3. Résultat attendu pour le linking lexical

On définit une structure pour stocker le résultat du matching lexical.

In [47]:
from dataclasses import dataclass
from typing import Dict, List


@dataclass
class LexicalLinkingResult:
    """
    Résultat de la baseline de schema linking.
    - matched_tables : liste des noms de tables détectées dans la question
    - matched_columns : liste des noms complets "table.colonne" détectés
    - token_to_tables : mapping idx_token -> tables candidates
    - token_to_columns : mapping idx_token -> colonnes candidates
    """
    matched_tables: List[str]
    matched_columns: List[str]
    token_to_tables: Dict[int, List[str]]
    token_to_columns: Dict[int, List[str]]


4. Fonction de baseline : matching lexical simple
Ici on utilise les lemmes des tokens de la question (étape 3) et on les compare aux noms de tables / colonnes.

In [49]:
def lexical_schema_linking(
    q_preproc: QuestionPreproc,
    norm_schema: NormalizedSchema
) -> LexicalLinkingResult:
    """
    Baseline de schema linking basée sur le matching lexical :
    - Pour chaque token (lemme) de la question :
        * on cherche s'il correspond à un nom de table
        * on cherche s'il correspond (ou apparaît) dans un nom de colonne
    - On accumule les tables/colonnes "matchées" et on garde
      une trace de quels tokens pointent vers quelles tables/colonnes.
    """

    matched_tables = set()    # ensemble de noms de tables
    matched_columns = set()   # ensemble de noms "table.colonne"

    token_to_tables: Dict[int, List[str]] = {}
    token_to_columns: Dict[int, List[str]] = {}

    # Parcours des tokens de la question prétraitée
    for tok in q_preproc.tokens:
        # On travaille avec le lemme en minuscule
        lemma = tok.lemma.lower()

        # On ignore les tokens vides ou très courts (ex: "a", "of"... éventuellement)
        if len(lemma) < 2:
            continue

        # ================
        # 1) Matching avec les tables
        # ================
        for tname in norm_schema.table_names:
            # Matching très simple :
            # - égalité exacte (lemma == nom de table)
            # - inclusion (lemma dans tname ou l'inverse)
            if lemma == tname:
                matched_tables.add(tname)
                token_to_tables.setdefault(tok.idx, []).append(tname)
            elif lemma in tname or tname in lemma:
                matched_tables.add(tname)
                token_to_tables.setdefault(tok.idx, []).append(tname)

        # ================
        # 2) Matching avec les colonnes
        # ================
        for col in norm_schema.columns:
            # On va découper le nom de la colonne en mots
            # ex: "budget in billions" -> ["budget", "in", "billions"]
            col_tokens = col.column_name.split()

            # Matching si le lemme est exactement un des mots de la colonne
            # ou si le lemme est égal au nom complet de la colonne.
            if lemma == col.column_name or lemma in col_tokens:
                matched_columns.add(col.full_name)
                token_to_columns.setdefault(tok.idx, []).append(col.full_name)

    return LexicalLinkingResult(
        matched_tables=sorted(matched_tables),
        matched_columns=sorted(matched_columns),
        token_to_tables=token_to_tables,
        token_to_columns=token_to_columns
    )


5. Exemple complet sur ton premier exemple

In [51]:
# ============================================
# Exemple d'utilisation – Étape 4
# ============================================

example = train_data[0]

question = example["question"]
db_id = example["db_id"]

print("Question originale :", question)
print("db_id :", db_id)

# 1) Récupérer le schéma brut
schema_raw = get_db_schema(db_id, tables)

# 2) Normaliser le schéma
norm_schema = normalize_schema(schema_raw)

# 3) Prétraiter la question (Étape 3 déjà faite)
q_preproc = preprocess_question(question, nlp)

# 4) Appliquer la baseline de schema linking
lex_links = lexical_schema_linking(q_preproc, norm_schema)

print("\n=== Résultat baseline Schema Linking (Étape 4) ===")
print("Tables détectées :", lex_links.matched_tables)
print("Colonnes détectées :")
for col in lex_links.matched_columns:
    print("  -", col)

print("\nDétails token -> tables :")
for idx, tables_list in lex_links.token_to_tables.items():
    tok = q_preproc.tokens[idx]
    print(f"  token '{tok.text}' (idx={idx}) → tables : {tables_list}")

print("\nDétails token -> colonnes :")
for idx, cols_list in lex_links.token_to_columns.items():
    tok = q_preproc.tokens[idx]
    print(f"  token '{tok.text}' (idx={idx}) → colonnes : {cols_list}")


Question originale : How many heads of the departments are older than 56 ?
db_id : department_management

=== Résultat baseline Schema Linking (Étape 4) ===
Tables détectées : ['department', 'head']
Colonnes détectées :
  - department.department id
  - head.head id
  - management.department id
  - management.head id

Détails token -> tables :
  token 'heads' (idx=2) → tables : ['head']
  token 'departments' (idx=5) → tables : ['department']

Détails token -> colonnes :
  token 'heads' (idx=2) → colonnes : ['head.head id', 'management.head id']
  token 'departments' (idx=5) → colonnes : ['department.department id', 'management.department id']


In [53]:
# On prend le 2ᵉ exemple (index 1) de train_spider
example2 = train_data[1]

question2 = example2["question"]
db_id2 = example2["db_id"]

print("Question originale :", question2)
print("db_id :", db_id2)

# 1) Récupérer le schéma brut
schema_raw2 = get_db_schema(db_id2, tables)

# 2) Normaliser le schéma
norm_schema2 = normalize_schema(schema_raw2)

# 3) Prétraiter la question (Étape 3)
q_preproc2 = preprocess_question(question2, nlp)

print("\nQuestion normalisée :", q_preproc2.normalized)
print("Tokens :")
for t in q_preproc2.tokens:
    print(f"  idx={t.idx:2d} text={t.text:10s} lemma={t.lemma:10s} pos={t.pos:5s} dep={t.dep:10s} head={t.head}")

print("\nValeurs candidates :", q_preproc2.values)

# 4) Appliquer la baseline de schema linking (Étape 4)
lex_links2 = lexical_schema_linking(q_preproc2, norm_schema2)

print("\n=== Résultat baseline Schema Linking pour cet exemple ===")
print("Tables détectées :", lex_links2.matched_tables)
print("Colonnes détectées :")
for col in lex_links2.matched_columns:
    print("  -", col)

print("\nDétails token -> tables :")
for idx, tables_list in lex_links2.token_to_tables.items():
    tok = q_preproc2.tokens[idx]
    print(f"  token '{tok.text}' (idx={idx}) → tables : {tables_list}")

print("\nDétails token -> colonnes :")
for idx, cols_list in lex_links2.token_to_columns.items():
    tok = q_preproc2.tokens[idx]
    print(f"  token '{tok.text}' (idx={idx}) → colonnes : {cols_list}")


Question originale : List the name, born state and age of the heads of departments ordered by age.
db_id : department_management

Question normalisée : list the name born state and age of the heads of departments ordered by age
Tokens :
  idx= 0 text=list       lemma=list       pos=VERB  dep=ROOT       head=list
  idx= 1 text=the        lemma=the        pos=DET   dep=det        head=name
  idx= 2 text=name       lemma=name       pos=NOUN  dep=npadvmod   head=born
  idx= 3 text=born       lemma=bear       pos=VERB  dep=amod       head=state
  idx= 4 text=state      lemma=state      pos=NOUN  dep=dobj       head=list
  idx= 5 text=and        lemma=and        pos=CCONJ dep=cc         head=state
  idx= 6 text=age        lemma=age        pos=NOUN  dep=conj       head=state
  idx= 7 text=of         lemma=of         pos=ADP   dep=prep       head=state
  idx= 8 text=the        lemma=the        pos=DET   dep=det        head=heads
  idx= 9 text=heads      lemma=head       pos=NOUN  dep=pobj     

Dans l’étape 4, nous avons mis en place une baseline de schema linking basée uniquement sur le matching lexical.
Pour chaque token de la question (via son lemme), nous comparons son contenu aux noms normalisés des tables et colonnes du schéma afin d’identifier les éléments susceptibles d’être mentionnés dans la question.
Cette étape fournit une première approximation des tables et colonnes pertinentes, servant de fondation aux méthodes plus avancées du schema linking dans l’étape 5.


Bien que l’étape 4 permette d’identifier des correspondances simples entre les lemmes de la question et les éléments du schéma, cette approche reste limitée face aux variations lexicales. L’étape 5 vient alors enrichir ce processus grâce à un NER schema-aware combinant analyse linguistique et fuzzy matching, permettant de détecter plus précisément les tables, colonnes et valeurs réellement mentionnées dans la question.

### Étape 5 : NER schema-aware avancé avec spaCy + fuzzy matching

In [55]:
@dataclass
class SchemaEntity:
    """
    Représente une entité détectée dans la question et liée au schéma.

    - label : type d'entité (ex: "TABLE", "COLUMN", "VALUE")
    - text : texte tel qu'il apparaît dans la question
    - start_idx : index du token de début (dans la question spaCy)
    - end_idx : index du token de fin (exclu)
    - linked_schema : nom de table / colonne correspondante dans le schéma
                      (ex: "head" ou "head.age"), ou None pour VALUE brute
    - score : score de similarité / confiance (entre 0 et 1)
    """
    label: str
    text: str
    start_idx: int
    end_idx: int
    linked_schema: Optional[str]
    score: float


Petite fonction utilitaire : meilleur match fuzzy

In [57]:
def fuzzy_best_match(cand: str, choices: List[str]) -> Tuple[Optional[str], float]:
    """
    Retourne la meilleure correspondance fuzzy (approx.) entre une chaîne 'cand'
    et une liste de 'choices' (noms de tables ou colonnes).

    On utilise rapidfuzz.fuzz.ratio, qui donne un score entre 0 et 100.
    On renvoie (élément_choisi, score).
    """
    best_item = None
    best_score = 0.0

    for ch in choices:
        s = fuzz.ratio(cand, ch)  # similarité de 0 à 100
        if s > best_score:
            best_score = s
            best_item = ch

    return best_item, best_score


Fonction principale : NER schema-aware

Idée :

On réutilise les valeurs déjà extraites dans l’étape 3 (q_preproc.values) → entités VALUE

Pour chaque nom (NOUN) ou nom propre (PROPN) de la question :

on cherche le meilleur match fuzzy parmi les tables

et parmi les colonnes

selon les scores, on décide si c’est plutôt une TABLE, une COLUMN, ou rien

Tu peux ajuster les seuils (table_threshold, column_threshold) en fonction de tes tests.

In [75]:
def schema_aware_ner(
    q_preproc: QuestionPreproc,
    norm_schema: NormalizedSchema,
    table_threshold: int = 80,
    column_threshold: int = 70
) -> List[SchemaEntity]:
    """
    Détecte des entités orientées schéma dans la question :
      - TABLE : token qui ressemble à un nom de table
      - COLUMN : token qui ressemble à un nom de colonne (ou partie de colonne)
      - VALUE : valeurs (nombres, strings) déjà extraites à l'étape 3

    Combinaison de :
      - analyse linguistique (POS, lemmes) de spaCy
      - fuzzy matching (rapidfuzz) avec les noms de tables/colonnes
    """

    entities: List[SchemaEntity] = []
    doc = q_preproc.doc  # doc spaCy issu de preprocess_question

    # ===========================
    # 1) Entités de type VALUE
    # ===========================
    for val in q_preproc.values:
        # On essaie de retrouver la position de cette valeur dans les tokens
        start_idx = -1
        for tok in q_preproc.tokens:
            if tok.text == val:
                start_idx = tok.idx
                break

        entities.append(
            SchemaEntity(
                label="VALUE",
                text=val,
                start_idx=start_idx,
                end_idx=start_idx + 1 if start_idx >= 0 else -1,
                linked_schema=None,   # pas encore liée à une colonne précise
                score=1.0             # confiance max : on a vu la valeur clairement
            )
        )

    # ===========================
    # 2) Entités TABLE / COLUMN
    # ===========================

    # Liste des noms de tables (ex: ["department", "head", "management"])
    table_candidates = norm_schema.table_names

    # Construction d'une liste enrichie de candidats colonnes
    # + d'une map pour retrouver le full_name ("table.colonne")
    column_candidates: List[str] = []
    column_map: Dict[str, str] = {}

    for c in norm_schema.columns:
        # 1) full name "table.colonne"
        column_candidates.append(c.full_name)
        column_map[c.full_name] = c.full_name

        # 2) nom complet de la colonne (ex: "budget in billions")
        column_candidates.append(c.column_name)
        column_map[c.column_name] = c.full_name

        # 3) mots individuels (ex: "budget", "billions")
        for piece in c.column_name.split():
            column_candidates.append(piece)
            column_map[piece] = c.full_name

    # On parcourt les tokens spaCy (doc) plutôt que q_preproc.tokens
    # pour avoir accès aux infos POS/dep complètes
    for tok in doc:
        # On ne s'intéresse qu'aux noms (NOUN) et noms propres (PROPN)
        if tok.pos_ not in ("NOUN", "PROPN","VERB"):
            continue

        lemma = tok.lemma_.lower()

        # On ignore les tout petits tokens (comme "a", "an")
        if len(lemma) < 2:
            continue

        # Meilleur match fuzzy avec les tables
        best_table, score_table = fuzzy_best_match(lemma, table_candidates)

        # Meilleur match fuzzy avec les colonnes ENRICHI
        best_column_key, score_column = fuzzy_best_match(lemma, column_candidates)

        # ===============================
        # Décision : TABLE vs COLUMN
        # ===============================

        # 1) TABLE si score bon et >= score colonne
        if best_table is not None and score_table >= table_threshold and score_table >= score_column:
            ent = SchemaEntity(
                label="TABLE",
                text=tok.text,
                start_idx=tok.i,
                end_idx=tok.i + 1,
                linked_schema=best_table,
                score=score_table / 100.0
            )
            entities.append(ent)

        # 2) sinon COLUMN si score colonne assez bon
        elif best_column_key is not None and score_column >= column_threshold:
            full_name = column_map[best_column_key]  # récupère "table.colonne"
            ent = SchemaEntity(
                label="COLUMN",
                text=tok.text,
                start_idx=tok.i,
                end_idx=tok.i + 1,
                linked_schema=full_name,
                score=score_column / 100.0
            )
            entities.append(ent)

        # Sinon, on ignore ce token (pas suffisamment ressemblant)

    return entities


In [77]:
# ===========================
# Exemple d'utilisation Étape 5
# ===========================

example = train_data[0]
question = example["question"]
db_id = example["db_id"]

print("Question originale :", question)
print("db_id :", db_id)

# Schéma brut + normalisation
schema_raw = get_db_schema(db_id, tables)
norm_schema = normalize_schema(schema_raw)

# Préprocessing (Étape 3)
q_preproc = preprocess_question(question, nlp)

# NER schema-aware (Étape 5)
schema_entities = schema_aware_ner(q_preproc, norm_schema)

print("\n=== Entités schema-aware détectées (Étape 5) ===")
for ent in schema_entities:
    print(f"- [{ent.label}] '{ent.text}' (tokens {ent.start_idx}-{ent.end_idx}) "
          f"→ linked_schema={ent.linked_schema}, score={ent.score:.2f}")


Question originale : How many heads of the departments are older than 56 ?
db_id : department_management

=== Entités schema-aware détectées (Étape 5) ===
- [VALUE] '56' (tokens 9-10) → linked_schema=None, score=1.00
- [TABLE] 'heads' (tokens 2-3) → linked_schema=head, score=1.00
- [TABLE] 'departments' (tokens 5-6) → linked_schema=department, score=1.00


In [79]:
question_test = "List the name and age of the heads in departments with budget in billions greater than 10"
db_id_test = "department_management"
schema_raw = get_db_schema(db_id_test, tables)
norm_schema = normalize_schema(schema_raw)
q_preproc_test = preprocess_question(question_test, nlp)
schema_entities_test = schema_aware_ner(q_preproc_test, norm_schema)

print("Question originale :", question_test)
print("Question normalisée :", q_preproc_test.normalized)
print("\n=== Entités schema-aware détectées (Étape 5) ===")
for ent in schema_entities_test:
    print(f"- [{ent.label}] '{ent.text}' (tokens {ent.start_idx}-{ent.end_idx}) "
          f"→ linked_schema={ent.linked_schema}, score={ent.score:.2f}")



Question originale : List the name and age of the heads in departments with budget in billions greater than 10
Question normalisée : list the name and age of the heads in departments with budget in billions greater than 10

=== Entités schema-aware détectées (Étape 5) ===
- [VALUE] '10' (tokens 16-17) → linked_schema=None, score=1.00
- [COLUMN] 'name' (tokens 2-3) → linked_schema=head.name, score=1.00
- [COLUMN] 'age' (tokens 4-5) → linked_schema=head.age, score=1.00
- [TABLE] 'heads' (tokens 7-8) → linked_schema=head, score=1.00
- [TABLE] 'departments' (tokens 9-10) → linked_schema=department, score=1.00
- [COLUMN] 'budget' (tokens 11-12) → linked_schema=department.budget in billions, score=1.00
- [COLUMN] 'billions' (tokens 13-14) → linked_schema=department.budget in billions, score=0.93


In [81]:
question_test = "List all departments created before the year 1990."
db_id_test = "department_management"
schema_raw = get_db_schema(db_id_test, tables)
norm_schema = normalize_schema(schema_raw)
q_preproc_test = preprocess_question(question_test, nlp)
schema_entities_test = schema_aware_ner(q_preproc_test, norm_schema)

print("Question originale :", question_test)
print("Question normalisée :", q_preproc_test.normalized)
print("\n=== Entités schema-aware détectées (Étape 5) ===")
for ent in schema_entities_test:
    print(f"- [{ent.label}] '{ent.text}' (tokens {ent.start_idx}-{ent.end_idx}) "
          f"→ linked_schema={ent.linked_schema}, score={ent.score:.2f}")



Question originale : List all departments created before the year 1990.
Question normalisée : list all departments created before the year 1990

=== Entités schema-aware détectées (Étape 5) ===
- [VALUE] '1990' (tokens 7-8) → linked_schema=None, score=1.00
- [TABLE] 'departments' (tokens 2-3) → linked_schema=department, score=1.00
- [COLUMN] 'created' (tokens 3-4) → linked_schema=department.creation, score=0.71


L’étape 5 réalise un repérage intelligent des éléments du schéma mentionnés dans la question (tables, colonnes et valeurs), en reliant les mots du langage naturel aux structures exactes de la base de données. Cela permet d’identifier quelles parties du schéma sont réellement pertinentes pour répondre à la question. Ces informations serviront ensuite à construire un input textuel riche et précis, qui guidera efficacement le modèle génératif (comme BART) dans la production de la requête SQL correcte.

### Étape 6 — Préparation et Structuration du Contexte Schéma–Question

Dans cette étape, nous rassemblons et organisons toutes les informations extraites des étapes précédentes afin de préparer la construction de l’entrée textuelle destinée au modèle génératif. Concrètement, l’étape 6 regroupe :

la question prétraitée (lemmatisation, valeurs numériques, tokens utiles) ;

le schéma normalisé associé à la base cible (tables, colonnes et clés étrangères) ;

les entités orientées schéma identifiées à l’étape 5 (tables, colonnes et valeurs pertinentes).

Cette structuration a pour objectif de filtrer et de mettre en forme uniquement les éléments du schéma réellement pertinents pour la génération SQL. L’étape 6 agit ainsi comme une couche d’intégration, assurant la transition entre l’analyse linguistique/sémantique (étapes 3 à 5) et la construction de l’input final pour le modèle Seq2Seq (étape 7).

In [116]:

@dataclass
class ModelContext:
    """
    Représente le contexte complet nécessaire pour la génération SQL :
      - question originale et prétraitée
      - schéma brut et normalisé
      - entités orientées schéma (TABLE, COLUMN, VALUE)
    """
    db_id: str
    question_original: str
    question_preproc: QuestionPreproc
    schema_raw: Dict[str, Any]
    schema_norm: NormalizedSchema
    schema_entities: List[SchemaEntity]


In [118]:
def build_model_context(
    example: Dict[str, Any],
    tables: List[Dict[str, Any]],
    nlp
) -> ModelContext:
    """
    Étape 6 : préparation et structuration du contexte schéma–question.

    À partir d'un exemple du dataset Spider, cette fonction :
      1) récupère le schéma brut correspondant (tables.json),
      2) normalise ce schéma (tables, colonnes, types, foreign keys),
      3) prétraite la question (Étape 3),
      4) applique le NER orienté schéma (Étape 5).

    Elle renvoie un objet ModelContext qui regroupe toutes ces informations
    et qui servira d'entrée à l'étape 7 (construction de l'input texte
    pour BART).
    """

    # 1) Récupération de la question et de l'identifiant de la DB
    question = example["question"]
    db_id = example["db_id"]

    # 2) Schéma brut à partir de tables.json
    schema_raw = get_db_schema(db_id, tables)

    # 3) Normalisation du schéma (noms de tables/colonnes, etc.)
    schema_norm = normalize_schema(schema_raw)

    # 4) Prétraitement de la question (Étape 3)
    q_preproc = preprocess_question(question, nlp)

    # 5) NER schema-aware (Étape 5)
    schema_entities = schema_aware_ner(q_preproc, schema_norm)

    # 6) Construction du contexte global
    ctx = ModelContext(
        db_id=db_id,
        question_original=question,
        question_preproc=q_preproc,
        schema_raw=schema_raw,
        schema_norm=schema_norm,
        schema_entities=schema_entities,
    )

    return ctx


In [120]:
# On prend un exemple du train Spider
example = train_data[0]

# Étape 6 : on construit le contexte
ctx = build_model_context(example, tables, nlp)

print("DB :", ctx.db_id)
print("Question originale :", ctx.question_original)
print("Question normalisée :", ctx.question_preproc.normalized)

print("\nTables normalisées :", ctx.schema_norm.table_names)

print("\nEntités détectées :")
for ent in ctx.schema_entities:
    print(f"- {ent.label} '{ent.text}' -> {ent.linked_schema}, score={ent.score:.2f}")


DB : department_management
Question originale : How many heads of the departments are older than 56 ?
Question normalisée : how many heads of the departments are older than 56

Tables normalisées : ['department', 'head', 'management']

Entités détectées :
- VALUE '56' -> None, score=1.00
- TABLE 'heads' -> head, score=1.00
- TABLE 'departments' -> department, score=1.00


In [122]:
from typing import List, Dict, Any

def build_model_input(
    question: str,
    db_id: str,
    q_preproc: QuestionPreproc,
    norm_schema: NormalizedSchema,
    schema_entities: List[SchemaEntity],
    schema_raw: Dict[str, Any],
) -> str:
    """
    Étape 7 : linéarise la question + le schéma en un texte structuré
    utilisable comme entrée pour un modèle Seq2Seq (ex: BART).

    On inclut :
      - la question normalisée,
      - les tables + colonnes,
      - les foreign keys (cas général),
      - les entités schema-aware (TABLE, COLUMN, VALUE).
    """

    lines: List[str] = []

    # 1) Infos générales
    lines.append(f"DB: {db_id}")
    lines.append(f"QUESTION: {q_preproc.normalized}")
    lines.append("")

    # 2) Tables + colonnes
    lines.append("TABLES:")
    for i, tname in enumerate(norm_schema.table_names):
        cols = [c.column_name for c in norm_schema.columns if c.table_idx == i]
        cols_str = ", ".join(cols) if cols else "(no columns)"
        lines.append(f"- {tname}: {cols_str}")
    lines.append("")

    # 3) Foreign keys (relations entre colonnes/tables)
    fk_list = schema_raw.get("foreign_keys", [])
    col_names_raw = schema_raw.get("column_names", [])
    table_names_raw = schema_raw.get("table_names", [])

    if fk_list:
        lines.append("FOREIGN_KEYS:")
        for src_idx, tgt_idx in fk_list:
            if not (0 <= src_idx < len(col_names_raw) and 0 <= tgt_idx < len(col_names_raw)):
                continue

            src_table_idx, src_col_name = col_names_raw[src_idx]
            tgt_table_idx, tgt_col_name = col_names_raw[tgt_idx]

            if src_table_idx == -1 or tgt_table_idx == -1:
                continue

            src_table_name = table_names_raw[src_table_idx]
            tgt_table_name = table_names_raw[tgt_table_idx]

            src_full = f"{src_table_name}.{src_col_name}"
            tgt_full = f"{tgt_table_name}.{tgt_col_name}"

            lines.append(f"- {src_full} -> {tgt_full}")
        lines.append("")

    # 4) Entités schema-aware (TABLE / COLUMN / VALUE)
    lines.append("ENTITIES:")
    for ent in schema_entities:
        lines.append(f"- {ent.label} '{ent.text}' -> {ent.linked_schema}")
    lines.append("")

    return "\n".join(lines)


The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [124]:
# ===========================
# Test Étapes 6 + 7 sur un exemple
# ===========================

example = train_data[0]

# Étape 6 : construire le contexte
ctx = build_model_context(example, tables, nlp)

# Étape 7 : construire l'input texte pour BART
model_input = build_model_input(
    question=ctx.question_original,
    db_id=ctx.db_id,
    q_preproc=ctx.question_preproc,
    norm_schema=ctx.schema_norm,
    schema_entities=ctx.schema_entities,
    schema_raw=ctx.schema_raw,
)

print("===== INPUT POUR BART =====")
print(model_input)
print("\n===== SQL CIBLE (Spider) =====")
print(example["query"])


===== INPUT POUR BART =====
DB: department_management
QUESTION: how many heads of the departments are older than 56

TABLES:
- department: department id, name, creation, ranking, budget in billions, num employees
- head: head id, name, born state, age
- management: department id, head id, temporary acting

FOREIGN_KEYS:
- management.head id -> head.head id
- management.department id -> department.department id

ENTITIES:
- VALUE '56' -> None
- TABLE 'heads' -> head
- TABLE 'departments' -> department


===== SQL CIBLE (Spider) =====
SELECT count(*) FROM head WHERE age  >  56


In [126]:
# ===========================
# Test Étapes 6 + 7 sur question_test
# ===========================

question_test = "List all departments created before the year 1990."
db_id_test = "department_management"

# On fabrique un "fake" exemple au même format que Spider
example_test = {
    "question": question_test,
    "db_id": db_id_test,
}

# Étape 6 : construire le contexte
ctx_test = build_model_context(example_test, tables, nlp)

# Étape 7 : construire l'input texte pour BART
model_input_test = build_model_input(
    question=ctx_test.question_original,
    db_id=ctx_test.db_id,
    q_preproc=ctx_test.question_preproc,
    norm_schema=ctx_test.schema_norm,
    schema_entities=ctx_test.schema_entities,
    schema_raw=ctx_test.schema_raw,
)

print("===== INPUT POUR BART (question_test) =====")
print(model_input_test)


===== INPUT POUR BART (question_test) =====
DB: department_management
QUESTION: list all departments created before the year 1990

TABLES:
- department: department id, name, creation, ranking, budget in billions, num employees
- head: head id, name, born state, age
- management: department id, head id, temporary acting

FOREIGN_KEYS:
- management.head id -> head.head id
- management.department id -> department.department id

ENTITIES:
- VALUE '1990' -> None
- TABLE 'departments' -> department
- COLUMN 'created' -> department.creation



In [128]:
###ÉTAPE 7 — Préparation du dataset pour entraîner BART

In [138]:
import json

def build_pairs_from_file(json_path: str, tables, nlp):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    pairs = []
    for example in data:
        ctx = build_model_context(example, tables, nlp)
        model_input = build_model_input(
            question=ctx.question_original,
            db_id=ctx.db_id,
            q_preproc=ctx.question_preproc,
            norm_schema=ctx.schema_norm,
            schema_entities=ctx.schema_entities,
            schema_raw=ctx.schema_raw,
        )
        gold_sql = example["query"]
        pairs.append({
            "input": model_input,
            "output": gold_sql,
            "db_id": ctx.db_id
        })
    return pairs


In [140]:
# Paires pour train_spider
pairs_train_spider = build_pairs_from_file("datafinal/train_spider.json", tables, nlp)

# Paires pour train_others
pairs_train_others = build_pairs_from_file("datafinal/train_others.json", tables, nlp)

# Paires pour dev
pairs_dev = build_pairs_from_file("datafinal/dev.json", tables, nlp)

# Train final = spider + others
pairs_train_full = pairs_train_spider + pairs_train_others

print(len(pairs_train_spider), len(pairs_train_others), len(pairs_train_full), len(pairs_dev))


7000 1659 8659 1034


In [142]:
import json

# Sauvegarder les paires d'entraînement (train_spider + train_others)
with open("bart_train_pairsfinal.json", "w", encoding="utf-8") as f:
    json.dump(pairs_train_full, f, ensure_ascii=False, indent=2)

# Sauvegarder les paires de validation (dev.json)
with open("bart_dev_pairsfinal.json", "w", encoding="utf-8") as f:
    json.dump(pairs_dev, f, ensure_ascii=False, indent=2)

print("Fichiers créés : bart_train_pairs.json et bart_dev_pairs.json")


Fichiers créés : bart_train_pairs.json et bart_dev_pairs.json
